In [1]:
using Rewrite
using Rewrite: PatternRule, EvalRule, Associative, Commutative

### Setting up Spec
Defines rules for simplifying `+`, `-`, `*` and `/`.

In [2]:
struct SimplificationSpec
    rules::Rules
    context::Context
end

In [3]:
function default_spec()
    @vars x y z
    equalities = [
        PatternRule(@term(x * y + x * z), @term(x * (y + z))),
        PatternRule(@term(y / x + z / x), @term((y + z) / x)),
        PatternRule(@term(x - y), @term(x + -y)),
        PatternRule(@term(x + -x), @term(zero(x))),
        PatternRule(@term(-(x + y)), @term(-y + -x)),
        PatternRule(@term(-(-x)), @term(x)),
        PatternRule(@term(-1 * x), @term(-x)),
        PatternRule(@term(-x * y), @term(-(x * y))),
        EvalRule(+),
        EvalRule(-),
        EvalRule(*),
        EvalRule(/)
    ]

    properties = [
        Associative(+),
        Commutative(+),
        Associative(*),
        Commutative(*)
    ]

    return SimplificationSpec(Rules(equalities), Context(properties))
end

SPEC = default_spec();

In [16]:
function simplify(term::Term)
    Rewrite.with_context(SPEC.context) do
       normalize(term, SPEC.rules)
    end
end;

### Simplification Experiments

In [17]:
@syms x y z;

In [50]:
[
    simplify(@term(x * y + x * z))
    simplify(@term(x * x + x * y + x * z))
    simplify(@term((x * x + x * y) + x * z))
]

3-element Array{Term,1}:
 @term(x * (z + y))          
 @term(x * x + x * y + x * z)
 @term(x * (z + (y + x)))    

In [51]:
[
    match(@term(x), @term(x)) |> !isempty
    match(@term(x), @term(y)) |> !isempty
    match(@term(x + y), @term(y + x)) |> !isempty
    match(@term(x + (y + z)), @term(x + y + z)) |> !isempty
]

4-element Array{Bool,1}:
  true
 false
  true
 false

The last example is a little concerning. We probably need more work on our rules, and more importantly a good specification of what types of expressions we need to handle.